In [ ]:
import numpy as np
import pandas as pd
import time as time
import os
import pickle as pkl
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt

# Functions used for creating tables and plots shown in the paper.  

In [ ]:
def CreateTablesTotReward(dictOfComb, dict_names):
    methods_names, names_to_use = [], []
    for met_name in list(dict_names.keys()):
        methods_names.append(met_name)
        names_to_use.append(dict_names[met_name])
    cols = ['R:0, C:0', 'R:1, C:0', 'R:2, C:0', 'R:0, C:1', 'R:1, C:1', 'R:2, C:1']
    rows_len, cols_len = len(methods_names) + 1, len(cols)
    toRet = [np.zeros((rows_len, cols_len)), np.zeros((rows_len, cols_len)), 
        np.zeros((rows_len, cols_len))]

    allExpNames  = list(dictOfComb.keys())

    for expName  in allExpNames:
        indexT = int(expName[0])
        indexErRev= int(expName[-3])
        indexErRow= int(expName[-1])
        colNum = indexErRev + indexErRow*3

        count_row = 0
        for method_name in methods_names:
            toRet[indexT][count_row, colNum] = np.round(
                dictOfComb[expName][method_name +'_avg_tot_reward'], 1)
            count_row += 1
        toRet[indexT][count_row, colNum] = np.round(
            dictOfComb[expName]['avg_best_offline'], 1)

    return toRet

def PlotRewardsOnTime(dictOfComb, dict_names, T_val, 
    indexT, window, nameToSave =None):
    methods_names, names_to_use = [], []
    for met_name in list(dict_names.keys()):
        methods_names.append(met_name)
        names_to_use.append(dict_names[met_name])
    fig,a =  plt.subplots(3,2)
    fig.set_size_inches(14, 18)
    nameRewardsOnTime = [name + '_rewards_over_time' for name in methods_names]
    allExpNames  = list(dictOfComb.keys())
    valBestMvAvg = {}

    posErrRow = [0.0, 0.1]
    posErrRev = [0.0, 0.1, 0.5]
    
    listCombErr = ['(' + str(iRev) + ', ' + str(iRow) + ')'             
                    for iRow in posErrRow
                    for iRev in posErrRev]
    
    for i in range(len(methods_names)):
        x = int(i/2)
        y = i % 2
        metDict = {}
        for expName  in allExpNames:
            if indexT == int(expName[0]):
                indexErRev= int(expName[-3])
                indexErRow= int(expName[-1])
                
                auxCumSum = (np.cumsum(dictOfComb[expName][nameRewardsOnTime[i]]))[:-window]
                arrayToUse =auxCumSum[window:] - auxCumSum[:-window]
                metDict['(' + str(posErrRev[indexErRev]) + ', ' + str(posErrRow[indexErRow]) + ')'] = arrayToUse
                if i == 0:
                    valBestMvAvg['(' + str(posErrRev[indexErRev]) + ', ' + str(posErrRow[indexErRow]) + ')'] = \
                        dictOfComb[expName]['avg_best_offline'] * window/T_val

        for combErr in listCombErr:
            a[x][y].plot(np.arange(len(metDict[combErr]))+window, metDict[combErr]/valBestMvAvg[combErr])
        a[2][1].set_axis_off()
        a[x][y].legend(listCombErr, prop={'size': 11})
        a[x][y].set_title(names_to_use[i])
        a[x][y].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    if nameToSave is not None:
        plt.savefig(nameToSave + '.png')
    plt.show()

def tableForFullLatex(tables, namesForLatex, allCombs, indexT = 2):
    for tabNumber, tabs in enumerate(tables):
        for row in range(5):
            print(namesForLatex[row] + ' & ' + str(allCombs[tabNumber][0]) + " $\\times$ " +\
                str(allCombs[tabNumber][1]) + ' & '+ \
                str(np.round((tabs[indexT][row,0]/tabs[indexT][5,0])*100, 1)) + "\% & "+ \
                str(np.round((tabs[indexT][row,1]/tabs[indexT][5,1])*100, 1)) + "\% & "+ \
                str(np.round((tabs[indexT][row,2]/tabs[indexT][5,2])*100, 1)) + "\% & "+ \
                str(np.round((tabs[indexT][row,3]/tabs[indexT][5,3])*100, 1)) + "\% & "+ \
                str(np.round((tabs[indexT][row,4]/tabs[indexT][5,4])*100, 1)) + "\% & "+ \
                str(np.round((tabs[indexT][row,5]/tabs[indexT][5,5])*100, 1)) + "\% \\\ \hline")
        print(' \hline')



## Create a Dictionary with the aggregated information obtained when running createAggregatedData.py

In [ ]:
aggResultsFolder = \
    '/Volumes/disk2s2/OnlineSetting/AggregatedResults/'

allCombs = [[5,5], [5,10], [10,5], [10,10], [25,25],  [25,50], [50,25], [50,50]]

dictAggPerComb = {}
for i in range(0,8):
    dictAggPerComb[str(allCombs[i][0])+'-'+str(allCombs[i][1])] = \
        pkl.load(open(aggResultsFolder + 'comb_' + str(i) + '.p', "rb"))

## Obtain the plots shown in the paper

Names as stored and how we want to show them on the graphs

In [ ]:
mets_names_w_subg = ['MatrixApp-subg', 'ThompsonSampling-subg', 'RidgeReg-subg',
    'RidgeRegPlusRandomness-subg', 'KnownThetaAst-subg'] #, 'FixTheta-subg'
names_to_show = ['Least Squares', 'Thompson Sampling', 'Ridge Regression',
    'Ridge Reg. + Uncertainty', 'Known Parameter'] #, 'Fix Parameter'
dict_names = {mets_names_w_subg[i]: names_to_show[i] for i in range(len(mets_names_w_subg))}

# Plots shown in the paper.

For obtaining other combinations of $d \times n$ simply change the pair of allCombs use. 

(We had allCombs = [(5,5), (5,10), (10,5), (10,10), (25,25),  (25,50), (50,25), (50,50)])

Plot when d =5, n = 10

In [ ]:
PlotRewardsOnTime(dictAggPerComb[str(allCombs[1][0])+'-'+str(allCombs[1][1])], dict_names,
    10000, 2, 250, nameToSave = 'MvAvg250_5_10')

Plot when d = n = 50

In [ ]:
PlotRewardsOnTime(dictAggPerComb[str(allCombs[7][0])+'-'+str(allCombs[7][1])], dict_names,
    10000, 2, 250, nameToSave = 'MvAvg250_50_50')

# Create full tables on Latex format

In [ ]:
tables = []
for i in range(0,8):
    tables.append(CreateTablesTotReward(dictAggPerComb[str(allCombs[i][0])+'-'+str(allCombs[i][1])], dict_names))
names_to_show_table = ['Least Squares', 'Thompson Sampling', 'Ridge Regression',
    'Ridge Reg. + Uncertainty', 'Known $\\theta^*$'] 


### Table for the case T = 1000

In [ ]:
tableForFullLatex(tables, names_to_show_table, allCombs, indexT= 0)

### Table for the case T = 5000

In [ ]:
tableForFullLatex(tables, names_to_show_table, allCombs, indexT= 1)

### Table for the case T = 10000

In [ ]:
tableForFullLatex(tables, names_to_show_table, allCombs, indexT= 2)